In [17]:
from ipywidgets import widgets
import numpy as np
import pandas as pd
import gmaps
import googlemaps
import os
# print(os.getcwd()) # es la carpeta jupyter
import sys
sys.path.append('../../') # secret.py lo cargará de la raíz del proyecto Data-Science
import secret
gmaps.configure(api_key=secret.GMAPS_KEY) # Your Google API key
#gmaps = googlemaps.Client(key=secret.GMAPS_KEY)

In [55]:
listing = pd.read_csv('../datasets/listings.csv')
#listing.head(10).to_excel('lista.xls')
listing = listing.drop(['host_id', 'host_name', 'neighbourhood_group', 'neighbourhood', 'calculated_host_listings_count', 'availability_365'], axis=1)

dict_listing = listing.head(3).to_dict(orient='index')
#dict_listing

localizaciones = [apartamento["location"] for apartamento in dict_listing]
info_box_template = """
<dl>
<dt>ID: {id}</dt><dd>{name}</dd>
<dt>Tipo propiedad</dt><dd>{property_type}</dd>
</dl>
"""

apartamento_info = [info_box_template.format(**apartamento) for apartamento in apartamentos]

marker_layer = gmaps.marker_layer(localizaciones, info_box_content=apartamento_info)
m = gmaps.Map()
m.add_layer(marker_layer)
m

#locations=(listing.head(10).values[:,6:8]).tolist()
#locations

#for location in locations:
#    print("loc" + str(location))

{0: {'id': 8207551,
  'last_review': '2015-09-27',
  'latitude': 41.3808442104594,
  'longitude': 2.163839677610885,
  'minimum_nights': 1,
  'name': 'Room in Sant Antoni huge terrasse',
  'number_of_reviews': 2,
  'price': 39,
  'reviews_per_month': 0.11,
  'room_type': 'Private room'},
 1: {'id': 14958299,
  'last_review': '2017-02-09',
  'latitude': 41.379712139441814,
  'longitude': 2.1644658002943986,
  'minimum_nights': 1,
  'name': 'Next To Las Ramblas, Unique, Wif1',
  'number_of_reviews': 15,
  'price': 60,
  'reviews_per_month': 2.26,
  'room_type': 'Entire home/apt'},
 2: {'id': 7766152,
  'last_review': '2015-11-16',
  'latitude': 41.3809633214534,
  'longitude': 2.165327414236724,
  'minimum_nights': 2,
  'name': 'Sunny whole flat in Raval/Ramblas',
  'number_of_reviews': 8,
  'price': 55,
  'reviews_per_month': 0.4,
  'room_type': 'Entire home/apt'}}

In [ ]:
# no funciona: module 'gmaps' has no attribute 'geocode'
# geocode_result = gmaps.geocode('1600 Amphitheatre Parkway, Mountain View, CA')

m = gmaps.Map()
m.add_layer(gmaps.heatmap_layer(locations))
m

In [46]:
apartamentos = [
    {"id": 8207551, "name": "Primero", "location": [41.3808442104594, 2.163839677610885], "property_type": "Apartamento"},
    {"id": 434332, "name": "Segundo", "location": [41.379712139441814, 2.1644658002943986], "property_type": "Casa"},
    {"id": 6676, "name": "Tercero", "location": [41.377832077541385, 2.1648009945320275], "property_type": "Chalet"}
]

localizaciones = [apartamento["location"] for apartamento in apartamentos]
info_box_template = """
<dl>
<dt>ID: {id}</dt><dd>{name}</dd>
<dt>Tipo propiedad</dt><dd>{property_type}</dd>
</dl>
"""

apartamento_info = [info_box_template.format(**apartamento) for apartamento in apartamentos]

marker_layer = gmaps.marker_layer(localizaciones, info_box_content=apartamento_info)
m = gmaps.Map()
m.add_layer(marker_layer)
m

In [40]:
nuclear_power_plants = [
    {"name": "Atucha", "location": (-34.0, -59.167), "active_reactors": 1},
    {"name": "Embalse", "location": (-32.2333, -64.4333), "active_reactors": 1},
]

plant_locations = [plant["location"] for plant in nuclear_power_plants]
info_box_template = """
<dl>
<dt>Name</dt><dd>{name}</dd>
<dt>Number reactors</dt><dd>{active_reactors}</dd>
</dl>
"""
plant_info = [info_box_template.format(**plant) for plant in nuclear_power_plants]

marker_layer = gmaps.marker_layer(plant_locations, info_box_content=plant_info)
m = gmaps.Map()
m.add_layer(marker_layer)
m